## Aromic scattering factors

source: https://henke.lbl.gov/optical_constants/asf.html

In [ ]:
from pathlib import Path
import pandas as pd
from mendeleev import element
from mendeleev.db import get_session
from mendeleev.models import ScatteringFactor, PropertyMetadata, ValueOrigin

In [ ]:
root = Path("../data/atomic-scattering-factors/")

## Preprocess the files

- some rows have traling tabs "\t" that break parsing
- a few file start with a comment or non standard header - manually correct

In [ ]:
proc = root.joinpath("preprocessed")
proc.mkdir(exist_ok=True)

for file in root.glob("*.nff"):

    with file.open("r") as fo:
        lines = fo.readlines()
    
    with proc.joinpath(file.name).open("w") as fo:
        for line in lines:
            fo.write(line.strip() + "\n")

## Load the data into the db

In [ ]:
session = get_session(read_only=False)

In [ ]:
for file in proc.glob("*.nff"):
    e = element(file.stem.capitalize())
    print(file.stem.capitalize(), e.atomic_number)
    df = pd.read_csv(file, sep="\t", header=0, engine="python", comment="#")
    print(df.columns)
    for index, row in df.iterrows():
        scattering_factor = ScatteringFactor(
            atomic_number=e.atomic_number,
            energy=row['E(eV)'],
            f1=row['f1'],
            f2=row['f2']
        )
        session.add(scattering_factor)
    session.commit()

## Test if the numbers of rows in the Scattering Factors table are the same as rows in the DF

In [ ]:
for file in proc.glob("*.nff"):
    e = element(file.stem.capitalize())
    print(file.stem.capitalize(), e.atomic_number)
    df = pd.read_csv(file, sep="\t", header=0, engine="python", comment="#")
    db_rows = session.query(ScatteringFactor).filter_by(atomic_number=e.atomic_number).count()
    
    if db_rows == df.shape[0]:
        print("OK")
    else:
        print(f"ERROR {db_rows=} {df.shape[0]=}")

## Create PropertyMetdata records

In [ ]:
ValueOrigin.STORED

In [ ]:
energy = PropertyMetadata(
    annotations="specific data references available at cited data source",
    attribute_name="energy",
    category="electric and optical properties",
    citation_keys="atomic_scattering_factors,henke1993xray",
    table_name="scattering_factors",
    class_name="ScatteringFactor",
    column_name="energy",
    description="Energy of the incident photon",
    unit="eV",
    value_origin=ValueOrigin.STORED,
)
session.add(energy)
session.commit()

In [ ]:
f1 = PropertyMetadata(
    annotations="specific data references available at cited data source",
    attribute_name="f1",
    category="electric and optical properties",
    citation_keys="atomic_scattering_factors,henke1993xray",
    table_name="scattering_factors",
    class_name="ScatteringFactor",
    column_name="f1",
    description="Scattering factor f1",
    unit=None,
    value_origin=ValueOrigin.STORED,
)
session.add(f1)
session.commit()

In [ ]:
f2 = PropertyMetadata(
    annotations="specific data references available at cited data source",
    attribute_name="f2",
    category="electric and optical properties",
    citation_keys="atomic_scattering_factors,henke1993xray",
    table_name="scattering_factors",
    class_name="ScatteringFactor",
    column_name="f2",
    description="Scattering factor f2",
    unit=None,
    value_origin=ValueOrigin.STORED,
)
session.add(f2)
session.commit()